<a href="https://colab.research.google.com/github/DamnScallion/Agents-Demo/blob/main/Agent_Demo_3_LangChain_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Agent Demo

This is a LangChain Agent Demo

## Setup

First, let's install our required packages and set our API keys:

In [ ]:
# Install packages
!pip install langchain openai tiktoken langchain-community chromadb pypdf -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.2 MB/s eta 0:00:00


### Setup OpenAI API Key

In [ ]:
# Set up OpenAI API Key (REPLACE with your own key)
import os
os.environ["OPENAI_API_KEY"] = "REPLACE THIS STRING WITH YOUR OPEN API KEY"

### Import LangChain libraries/modules

In [ ]:
# Import standard LangChain modules
from langchain.agents import initialize_agent, Tool, AgentType, load_tools
from langchain.llms import OpenAI
from langchain.chains import LLMChain, LLMMathChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.tools.retriever import create_retriever_tool
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, create_retrieval_chain
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

### Initialize OpenAI LLM

In [ ]:
# Initialize OpenAI LLM
llm = OpenAI(temperature=0.0001)

<ipython-input-5-d6eb68d2ddb7>:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.0001)


### Langchain.agents all the tool names

In [ ]:
from langchain.agents import get_all_tool_names
get_all_tool_names()

['sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-books',
 'google-lens',
 'google-serper',
 'google-scholar',
 'google-finance',
 'google-trends',
 'google-jobs',
 'google-serper-results-json',
 'searchapi',
 'searchapi-results-json',
 'serpapi',
 'dalle-image-generator',
 'twilio',
 'searx-search',
 'merriam-webster',
 'wikipedia',
 'arxiv',
 'golden-query',
 'pubmed',
 'human',
 'awslambda',
 'stackexchange',
 'sceneXplain',
 'graphql',
 'openweathermap-api',
 'dataforseo-api-search',
 'dataforseo-api-search-json',
 'eleven_labs_text2speech',
 'google_cloud_texttospeech',
 'read_file',
 'reddit_search',
 'news-api',
 'tmdb-api',
 'podcast-api',
 'memorize',
 'llm-math',
 'open-meteo-api',
 'requests',
 'requests_get',
 'requests_post',
 'requests_patch',
 'requests_put',
 'requests_delete',
 'terminal']

### Agents
- Zero-shot-agent (Zero-shot-agent could not remember memories)
- ReAct Conversational Agent (Conversational Agent allows dialogue and remmber previous chat history)

### Tool
- Math Tool
- Language Tool
- Retriever Tool

## Example 1: Math Tool Agent (Zero-shot)

In [ ]:
# Example 1: Math Tool Agent (Zero-shot)
tools = load_tools(["llm-math"], llm=llm)
math_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

<ipython-input-7-f6af386e05f2>:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  math_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [ ]:
# Run the agent
math_agent.run("What is (4.5 * 2.1)^2.2?")

<ipython-input-8-2505134ab57a>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  math_agent.run("What is (4.5 * 2.1)^2.2?")




> Entering new AgentExecutor chain...
 I should use a calculator to solve this problem.
Action: Calculator
Action Input: (4.5 * 2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer.
Final Answer: 139.94261298333066

> Finished chain.


'139.94261298333066'

### Example 2: Language Tool

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
# Simple LLM chain using a prompt
prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

<ipython-input-11-8a5533dae036>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
llm_tool = Tool(
    name="Language Model",
    func=llm_chain.run,
    description="General purpose LLM queries"
)

tools.append(llm_tool)

## Example 2: Zero-shot Agent

In [ ]:
zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)

In [ ]:
zero_shot_agent("What is the capital of Australia? What is the number of population of this capital divided by 2?")

<ipython-input-14-af78a97c0d3f>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  zero_shot_agent("What is the capital of Australia? What is the number of population of this capital divided by 2?")




> Entering new AgentExecutor chain...
 We can use a language model to find the capital of Australia and then use a calculator to divide the population by 2.
Action: Language Model
Action Input: "What is the capital of Australia?"
Observation: 

The capital of Australia is Canberra.
Thought: Now we can use a calculator to divide the population by 2.
Action: Calculator
Action Input: 410000
Observation: Answer: 410000
Thought: I now know the final answer.
Final Answer: The population of Canberra divided by 2 is 205000.

> Finished chain.


{'input': 'What is the capital of Australia? What is the number of population of this capital divided by 2?',
 'output': 'The population of Canberra divided by 2 is 205000.'}

## Example 3: ReAct Agent

- conversational-react-description agent
- Benifit: have memory can memory the previous chat history

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
conversational_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    max_iterations=3
)

In [ ]:
result = conversational_agent.run({"input": "What are the GDP of Australia over the past 5 years? Then divide it by the GDP of Germany?", "chat_history":[]})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Language Model
Action Input: The GDP of Australia over the past 5 years divided by the GDP of Germany
Observation:  over the past 5 years

The GDP of Australia over the past 5 years: 
2016: $1.2 trillion
2017: $1.3 trillion
2018: $1.4 trillion
2019: $1.4 trillion
2020: $1.4 trillion

Total GDP over 5 years: $6.7 trillion

The GDP of Germany over the past 5 years:
2016: $3.5 trillion
2017: $3.7 trillion
2018: $3.9 trillion
2019: $3.9 trillion
2020: $3.8 trillion

Total GDP over 5 years: $18.8 trillion

The ratio of Australia's GDP to Germany's GDP over the past 5 years is:
$6.7 trillion / $18.8 trillion = 0.356

Therefore, the GDP of Australia over the past 5 years is approximately 35.6% of the GDP of Germany over the past 5 years.
Thought: Do I need to use a tool? No
AI: The ratio of Australia's GDP to Germany's GDP over the past 5 years is approximately 35.6%. Is there anything else I can assist you 

In [ ]:
result = conversational_agent.run("What would be the result if the above result add the GDP of France?")



> Entering new AgentExecutor chain...
AI: The result would be the total GDP of the four countries (Spain, France, Portugal, and France) combined. Is there anything else I can assist you with?

> Finished chain.


## Example 4: Agentic RAG

Agentic Retrieval-Augmented Generation (Agentic RAG) is an emerging AI paradigm where large language models (LLMs) autonomously plan their next steps while pulling information from external sources. Unlike static retrieval-then-read patterns, Agentic RAG involves iterative calls to the LLM, interspersed with tool or function calls and structured outputs. The system evaluates results, refines queries, invokes additional tools if needed, and continues this cycle until a satisfactory solution is achieved.

### Load source documents

In [ ]:
loader = DirectoryLoader('./', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

### Create RAG knowledge base

In [ ]:
# embedding = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory="db")

vectordb.persist()

<ipython-input-74-9b140d77dfaa>:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
retriever = vectordb.as_retriever()

### Create RAG Retriever Tool

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "RAG_search",
    "Search for information about Strata domain knowledge. For any questions about Strata Inspection Report, you must use this tool"
)

In [ ]:
tools.append(retriever_tool)

In [ ]:
conversational_agent = initialize_agent(
    agent = 'conversational-react-description',
    tools = tools,
    llm = llm,
    verbose = True,
    memory = memory,
    max_iterations = 3
)

In [ ]:
result = conversational_agent.run("What is Motion 10, Open Discussion in the strata inspection report? Use tools if needed")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: RAG_search
Action Input: Motion 10, Open Discussion
Observation:  Motion 9: EV charging station 
That the Strata Committee revisit the possibility for the installation of EV charging infrastructure 
on either common property or authority for lot owners to install EV charging infrastructure 
within their car space (on specific conditions). 
 
That the strata manager is to investigate the cost of installation of EV chargers and provide an 
update to the committee. 
 Motion 10: Open Discussion

Motion is to be reviewed at next committee meeting. 
 Motion 9: Fire panel malfunction 
That the fire safety panel has malfunction causing several false alarms within a short period of 
time. Strata committee to review the situation and determine appropriate course of action and 
process. 
 
That the building manager confirms the panel has been repaired. 
 
The fire panel malfunction results in false alarm goin

In [ ]:
result = conversational_agent("What is Motion 9, EV charging station in the strata inspection report? Use Tool if needed")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: RAG_search
Action Input: Motion 9, EV charging station
Observation:  Motion 9: EV charging station 
That the Strata Committee revisit the possibility for the installation of EV charging infrastructure 
on either common property or authority for lot owners to install EV charging infrastructure 
within their car space (on specific conditions). 
 
That the strata manager is to investigate the cost of installation of EV chargers and provide an 
update to the committee. 
 Motion 10: Open Discussion

Strata Inspection Report:  SP 93491 Lot 208 
 
    
Nick Lazaris   |   0404 295 839   |   hello@yourstratasolutions.com.au 8 
 
Meriton informs that defects rectification is delayed due to constant wet weather, however they 
are still working to complete all defects as per agreement with the Owners Corporation. 
 Motion 9: EV charging station 
That the Strata Committee revisit the possibility for the install